In [3]:
import cv2
cap =cv2.VideoCapture(0) #0 is index of camera
while True : #infinite loop for video capture
    ret,frame= cap.read()
    if not ret:
        print("Error: Failed to capture frame from camera")
        break
    cv2.imshow("Webcam", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 
cap.release()
cv2.destroyAllWindows()

In [4]:
def play_video(video_path):
    cap =cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: could not open video file")
        return
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        cv2.imshow("Video",frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break 
    cap.release()
    cv2.destroyAllWindows()
video_path = (r"C:\Users\AMZ PC\OneDrive\Desktop\CV\Smoke detection\3826855-hd_1920_1080_24fps.mp4")
play_video(video_path)


In [5]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical

training_data=[r"C:\Users\AMZ PC\OneDrive\Desktop\CV\Smoke detection\fire_dataset\fire_images", 
               r"C:\Users\AMZ PC\OneDrive\Desktop\CV\Smoke detection\fire_dataset\non_fire_images"]

def load_images(training_data):
    images = []
    labels = []
    for i in range(len(training_data)):
        folder = training_data[i]
        label = i  # 0 for fire, 1 for non-fire
        for filename in os.listdir(folder):
            try:
                img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    # Resize image to 48x48 pixels
                    img = cv2.resize(img, (48, 48))
                    images.append(img)
                    labels.append(label)  # Correct label assignment
                else:
                    print(f"Warning: Failed to load {filename}")
            except Exception as e:
                print(f"Error: loading image {os.path.join(folder, filename)}: {e}")
    
    # Convert lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    # Debugging print statements to check data
    print(f"Total images loaded: {len(images)}")
    print(f"Images shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")
    print(f"Sample labels: {labels[:10]}")  # Print first 10 labels for debugging

    return images, labels

# Load images and labels
images, labels = load_images(training_data)

# Further Debugging
print(f"Type of images array: {type(images)}")
print(f"Type of labels array: {type(labels)}")
print(f"First image shape: {images[0].shape}")  # Check shape of first image


Total images loaded: 998
Images shape: (998, 48, 48)
Labels shape: (998,)
Sample labels: [0 0 0 0 0 0 0 0 0 0]
Type of images array: <class 'numpy.ndarray'>
Type of labels array: <class 'numpy.ndarray'>
First image shape: (48, 48)


In [6]:
#preprocessing
x_train, x_test , y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
x_train= x_train.reshape(x_train.shape[0],48,48,1).astype('float32')/255
x_test= x_test.reshape(x_test.shape[0],48,48,1).astype('float32')/255

y_train = to_categorical(y_train) 
y_test = to_categorical(y_test)

model=Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

c:\Users\AMZ PC\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [7]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=1, validation_data=(x_test, y_test))
model.save("fire_detection.model.h5")

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 322ms/step - accuracy: 0.6951 - loss: 1.1266 - val_accuracy: 0.7300 - val_loss: 0.5315
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 257ms/step - accuracy: 0.7651 - loss: 0.5449 - val_accuracy: 0.7350 - val_loss: 0.4592
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 249ms/step - accuracy: 0.7931 - loss: 0.4684 - val_accuracy: 0.8600 - val_loss: 0.3960
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 254ms/step - accuracy: 0.8383 - loss: 0.3671 - val_accuracy: 0.8600 - val_loss: 0.3142
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 246ms/step - accuracy: 0.8173 - loss: 0.3539 - val_accuracy: 0.8500 - val_loss: 0.3238
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 248ms/step - accuracy: 0.8362 - loss: 0.3315 - val_accuracy: 0.8700 - val_loss: 0.3089
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 245ms/step - accuracy: 0.8498 - loss: 0.2968 - val_accuracy: 0.8650 - val_loss: 0.3052
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 255ms/step - accuracy: 0.8523 - loss: 0.2770 - val_accuracy: 0.

In [16]:
from keras.models import load_model
model = load_model("fire_detection.model.h5")
def detect_fire(frame, threshold=0.5):
    preprocess_frame= cv2.cvtColor(cv2.resize(frame,(48,48)), cv2.COLOR_BGR2GRAY)
    preprocess_frame= np.expand_dims(preprocess_frame, axis=0)
    preprocess_frame= np.expand_dims(preprocess_frame, axis=-1)
    preprocess_frame= preprocess_frame.astype("float32")/255

    prediction =model.predict(preprocess_frame)
    if prediction[0][1]>= threshold:
        return True
    else:
        return False
cap=cv2.VideoCapture(r"C:\Users\AMZ PC\OneDrive\Desktop\CV\Smoke detection\3826855-hd_1920_1080_24fps.mp4")
if not cap.isOpened():
    print("Error: could not open video file")
    exit()

while True :
    ret, frame = cap.read()
    if not ret:
        break
    if detect_fire(frame):
        cv2.rectangle(frame, (100,100),(frame.shape[1]-100, frame.shape[0]-100), (0,0,255),2)
        cv2.putText(frame, "Warning, fire is detected", (50,50), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1 , (0,0,255), 2, cv2.LINE_AA)

    cv2.imshow("Video", frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()    

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━